### Importing

In [38]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/new_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['Unnamed: 0', 'id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y.loc[:, 'choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [39]:
X.sample(5)

,od,origin,destination,flight_departure_datetime,purchase_datetime,time_diff,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_price/PREF_price,ADVS_capacity,PREF_capacity,ADVS_inventory,PREF_inventory,advs_inv_full,pref_inv_full
87952,BBBCCC,B,D,2024-05-05 10:00,2024-04-29 11:27,513180.0,0,2,2,68,102,0.666667,250,50,231,37,1,1
4368,BBBCCC,B,D,2024-01-07 10:00,2023-12-20 05:39,1570860.0,0,2,1,52,73,0.712329,250,50,241,36,1,1
59451,CCCEEE,B,D,2024-03-25 07:00,2024-03-13 13:53,1012020.0,0,0,1,52,86,0.604651,250,50,217,3,1,1
38578,CCCDDD,B,D,2024-02-24 19:00,2024-02-20 22:59,331260.0,1,0,3,21,70,0.300000,250,50,195,0,1,0
171115,CCCEEE,B,D,2024-08-31 07:00,2024-07-30 08:39,2758860.0,0,2,1,30,78,0.384615,250,50,222,11,1,1


In [31]:
X.columns

Index(['od', 'origin', 'destination', 'flight_departure_datetime',
       'purchase_datetime', 'time_diff', 'trip_type', 'branded_fare',
       'number_of_pax', 'ADVS_price', 'PREF_price', 'ADVS_price/PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory',
       'advs_inv_full', 'pref_inv_full'],
      dtype='object')

### Preprocessing

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [7]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['origin', 'destination', 'od']),
    ('num', num_pipeline, ['time_diff', 'trip_type', 'branded_fare', 'number_of_pax', 'ADVS_price', 'PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory', 'advs_inv_full', 'pref_inv_full'])
  ], remainder='drop')

In [8]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

In [9]:
X_train_prep[0]

array([ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  3.26613535, -0.50960013,  0.99991145,
        0.28492134,  0.58777589,  1.16080662, -1.38176713, -1.12804817,
       -1.217205  ,  0.58043873,  0.        ,  0.36945853])

### Training

In [12]:
from xgboost import XGBClassifier

bst = XGBClassifier(learning_rate=.3, objective='multi:softmax')
bst.fit(X_train_prep, y_train)


y_pred = bst.predict(X_test_prep)
accuracy_score(y_pred, y_test)

0.7487792207792208

### Tuning

In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],           # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.2],         # Step size shrinkage
    'max_depth': [3, 5, 7],                    # Maximum depth of the trees
    'min_child_weight': [1, 3, 5],             # Minimum sum of instance weight needed in a child
    'subsample': [0.6, 0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0],       # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction to make a further partition on a leaf node
}

rnd_search = GridSearchCV(bst,
    param_grid=param_grid, verbose=3, cv=3, n_jobs=-1,
    scoring='neg_root_mean_squared_error')

rnd_search.fit(X_train_prep, y_train)

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


KeyboardInterrupt: 

### Predicting

In [17]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id', 'advs_inv_full', 'pref_inv_full']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = bst.predict(X_to_predict_prep)

/tmp/ipykernel_224944/548518805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = bst.predict(X_to_predict_prep)


In [18]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

/tmp/ipykernel_224944/1816271922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})


In [27]:
y_to_predict[y_to_predict['advs_inv_full'] == 0]

,id,advs_inv_full,pref_inv_full,choice


In [15]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [16]:
pd.DataFrame.to_csv(y_final, '../predictions/pred3.csv', index=False)